In [ ]:
"""
@File    : lab_v1.ipynb
@Create  : 2024/12/13 16:11:37
@Modify  : 2024/12/13 16:11:37
@Author  : Flora Sun
@Contact : floras@animocabrands.com
@Desc    : None
"""


In [1]:
cd '/Users/floras/Desktop/unique-user'

/Users/floras/Desktop/unique-user


In [65]:
import pandas as pd
import numpy as np
import os
import importlib
import sybil_functions
from collections import Counter
from concurrent.futures import ThreadPoolExecutor
import networkx as nx
from cdlib import algorithms
import networkx as nx
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import OPTICS
from sklearn.ensemble import IsolationForest
import parameters


importlib.reload(sybil_functions)
import trino_fetch_results
importlib.reload(trino_fetch_results)
importlib.reload(parameters)
import utility_functions
importlib.reload(utility_functions)
import api_functions
importlib.reload(api_functions)


from utility_functions import (
    read_from_multiple_csv,
    stretched_sigmoid,
    file_name_lst,
)

from api_functions import (
    check_contract,
)

from trino_fetch_results import (
    check_frequent_wallets,
    wallets_monthly_info,
)

from parameters import trino_data_column_names


# Import Data

In [3]:
# core wallet df
core_wallet_df = pd.read_excel(
    "/Users/floras/Desktop/unique-user/moca-airdrop/data/moca_airdrop_user_info.xlsx"
)
core_wallet_df.head()

,email,uuid,uuid.1,moca_id,id,name,abstract_account_address,web3auth_wallet_address,address,id.1,uuid.2,user_id
0,NaN,0016e1a3-0cdf-475b-b921-2d121453bece,0016e1a3-0cdf-475b-b921-2d121453bece,kppytiuynykiboo,7418689.0,kppytiuynykiboo,0x4b5591379a65c0180103f421cce640c679d1d9d8,0xc0f32955a9b37e6c1fa235a4f330c2d23545af88,0xc0f32955a9b37e6c1fa235a4f330c2d23545af88,590112.0,2d45d118-3d6f-4c7f-930d-5c9f3d56f612,7418689.0
1,semidmitrii@gmail.com,00284435-f796-45ad-8c61-5cf4a5417043,00284435-f796-45ad-8c61-5cf4a5417043,therock,25939.0,therock,0x3f1f1694941ebea878fe9e88b048dcb6f0c8f07b,0x5335b140366404b2bd875339322de16f9cd9651c,0xe23308c15434f903825112035cc1d26914fab078,60816.0,6ee964db-f14c-49ad-84db-84d937463cb3,25939.0
2,NaN,0048b42e-7630-42a2-9633-9ee78c35f2eb,0048b42e-7630-42a2-9633-9ee78c35f2eb,mocae,7361874.0,mocae,0x38c24c4158033ef3967f102748cba1d56079bd31,0x9c590bb4e0ffb4c9adfc03dfa2ee2484502ce067,0x9c590bb4e0ffb4c9adfc03dfa2ee2484502ce067,43567.0,5cbe41a2-b90c-43cb-9560-af5cb0cb2b2f,7361874.0
3,NaN,00799be0-8c24-4f0b-a442-bbb1279a63d3,00799be0-8c24-4f0b-a442-bbb1279a63d3,accelerando,8000019.0,accelerando,0xd86b10033c4d718b9b5f330ab270cd17c31cba97,0xaa6c6d0ed231da328814c2b7853d2e4350b70000,0xaa6c6d0ed231da328814c2b7853d2e4350b70000,294969.0,182f98d8-9098-4763-be3d-4e47985beb8f,8000019.0
4,NaN,00799be0-8c24-4f0b-a442-bbb1279a63d3,00799be0-8c24-4f0b-a442-bbb1279a63d3,accelerando,8000019.0,accelerando,0xd86b10033c4d718b9b5f330ab270cd17c31cba97,0xaa6c6d0ed231da328814c2b7853d2e4350b70000,0xd1f01c4bc479967f4dbde0a73f961dd14afa524f,343872.0,abe31f57-dcd4-4bec-a434-414e2c0d2181,8000019.0


In [4]:
# Get core wallet list
core_wallet_lst = core_wallet_df["address"].tolist()
# Get core wallet set
core_wallet_set = set(core_wallet_lst)

In [5]:
# Import transaction data
native_trans_path = "/Users/floras/Desktop/unique-user/moca-airdrop/data/raw-data"
native_trans_file_lst = file_name_lst(
    native_trans_path, startswith="native_m1", endswith=".csv"
)

In [6]:
native_trans_df_0 = read_from_multiple_csv(native_trans_path, native_trans_file_lst)
native_trans_df_0.columns = [
    "timestamp",
    "from_address",
    "to_address",
    "tx_count",
]
# remove self transactions
native_trans_df_0 = native_trans_df_0[
    native_trans_df_0["from_address"] != native_trans_df_0["to_address"]
]
print(native_trans_df_0.shape)
native_trans_df_0.head()

(28137, 4)


,timestamp,from_address,to_address,tx_count
0,2024-01-01 00:00:00.000 UTC,0xf3a8ae48c5029aab66f65be26ece0ff43bd1486f,0x25c805eaa831a0c98f45580e50676e041dcc87b8,1
1,2024-01-01 00:00:00.000 UTC,0xc888aea684b1d8f3b832f356dfc84bf0a1dc3406,0x9347ea27d20da04fa3af9444ebd086c73ff6553b,2
2,2024-01-01 00:00:00.000 UTC,0xb6f7f9326d631133c2b16e8822e6320ccfc34e23,0x852c2bd9230dcbf2a704d36b96a33c45e07ca192,1
3,2024-01-01 00:00:00.000 UTC,0x6bf7891adc8822aa26de1665887cccf921d475b8,0xd4118ba3793e1c20d6136bee2539ca011691d010,2
4,2024-01-01 00:00:00.000 UTC,0x077d360f11d220e4d5d831430c81c26c9be7c4a4,0x1f1bab9c578d92a10e2a061f86a9e6f74f25a8b5,1


# remove contract/hot wallets & transactions

## Define wallets to be checked based on the number of interacted wallets

In [7]:
def columns_unique_items(df, column_name_lst):
    unique_items = []
    for column_name in column_name_lst:
        unique_items.extend(df[column_name].unique().tolist())
    return list(set(unique_items))

In [8]:
all_wallets = columns_unique_items(native_trans_df_0, ["from_address", "to_address"])

In [9]:
def columns_item_count(unique_pairs_df, w_cols=["wallet_a", "wallet_b"]):
    # Flatten the selected columns into a single list
    wallet_z = unique_pairs_df[w_cols].values.flatten()
    # Count occurrences of each wallet using Counter
    wallet_z_count = Counter(wallet_z)
    # Convert the Counter object into a DataFrame
    wallet_z_count_df = pd.DataFrame(
        wallet_z_count.items(), columns=["wallet", "interacted_wallets"]
    )
    # Return the resulting DataFrame
    return wallet_z_count_df

In [10]:
def columns_item_unique_pairs(
    raw_transaction_df, address_cols=["from_address", "to_address"]
):
    # Ensure input columns are present in the DataFrame
    if not all(col in raw_transaction_df.columns for col in address_cols):
        raise ValueError(f"Columns {address_cols} not found in the DataFrame.")

    # Convert the columns to strings to ensure consistent dtype
    raw_transaction_df[address_cols] = raw_transaction_df[address_cols].astype(str)
    # Create sorted pairs using numpy operations (faster than apply)
    pairs = np.sort(raw_transaction_df[address_cols].to_numpy(dtype=str), axis=1)
    # Deduplicate pairs using numpy.unique
    unique_pairs = np.unique(pairs, axis=0)
    # Create a DataFrame for unique pairs
    unique_pairs_df = pd.DataFrame(unique_pairs, columns=["wallet_a", "wallet_b"])

    return unique_pairs_df

In [11]:
def columns_item_unique_pair_counts(
    raw_transaction_df, address_cols=["from_address", "to_address"]
):
    unique_pairs_df = columns_item_unique_pairs(raw_transaction_df, address_cols)
    # Use the optimized columns_item_count function
    wallet_count = columns_item_count(unique_pairs_df, w_cols=["wallet_a", "wallet_b"])
    # Reset index before returning
    wallet_count.reset_index(drop=True, inplace=True)
    return wallet_count

In [12]:
sample_wallet_pair_count_df = columns_item_unique_pair_counts(native_trans_df_0)
sample_wallet_pair_count_df.head()

,wallet,interacted_wallets
0,0x00000002f32c0886ee65d68059fbdb76ef6a6996,5
1,0x04f23cfd959b1f8aaf492e1d44c4c567102cf7db,3
2,0x7fd06d484a0d6ae974c3b45e1e63ff9ee5160889,3
3,0x8888889e0adf6c5afe2d5a6af91d31984365a1aa,3
4,0xe43888320d5361ce4357d0dccdb2aee4ad99b9a0,20


In [13]:
check_contract_wallet_lst = sample_wallet_pair_count_df[
    sample_wallet_pair_count_df["interacted_wallets"] > 1
]["wallet"].tolist()
check_contract_wallet_lst[:5]

['0x00000002f32c0886ee65d68059fbdb76ef6a6996',
 '0x04f23cfd959b1f8aaf492e1d44c4c567102cf7db',
 '0x7fd06d484a0d6ae974c3b45e1e63ff9ee5160889',
 '0x8888889e0adf6c5afe2d5a6af91d31984365a1aa',
 '0xe43888320d5361ce4357d0dccdb2aee4ad99b9a0']

In [14]:
def refine_lst(lst, exclude_lst):
    exclude_set = set(exclude_lst)
    return [x for x in lst if x not in exclude_set]

In [15]:
# remove core wallets from check_contract_wallet_lst
check_contract_wallet_lst = refine_lst(check_contract_wallet_lst, core_wallet_lst)

## get contract wallets list

In [16]:
def check_contract_wallet_addresses(wallet_addresses, max_threads=10):
    contract_wallets = []
    with ThreadPoolExecutor(max_threads) as executor:
        # Submit tasks for concurrent execution
        futures = [
            executor.submit(check_contract, wallet) for wallet in wallet_addresses
        ]
        for future in futures:
            results = future.result()
            if results:
                contract_wallets.append(results)
    return contract_wallets

In [17]:
check_contract_wallet_addresses(check_contract_wallet_lst[:10])

0x4d9339dd97db55e3b9bcbe65de39ff9c04d1c2cd is a contract on etherscan


['0x4d9339dd97db55e3b9bcbe65de39ff9c04d1c2cd']

## get hot wallets list

In [18]:
check_frequent_wallets(
    check_contract_wallet_lst[10:15], start_date_="2024-12-01", end_date_="2024-12-02"
)

['0x3b5a23f6207d87b423c6789d2625ea620423b32d',
 '0xdd3cb5c974601bc3974d908ea4a86020f9999e0c']

## remove contract/hot transactions

In [19]:
def filter_blacklist_transactions(df, col_lst, contract_lst):
    for col in col_lst:
        df = df[~df[col].isin(contract_lst)]
    return df

# Define weight

In [20]:
def columns_item_unique_pair_sums(
    raw_transaction_df, address_cols=["from_address", "to_address"], sum_col="tx_count"
):
    raw_transaction_df["pair"] = raw_transaction_df[address_cols].apply(
        lambda x: tuple(sorted(x)), axis=1
    )
    unique_pair_sums = raw_transaction_df.groupby("pair")[sum_col].sum().reset_index()
    unique_pair_sums["node_a"] = unique_pair_sums["pair"].apply(lambda x: x[0])
    unique_pair_sums["node_b"] = unique_pair_sums["pair"].apply(lambda x: x[1])
    unique_pair_sums.drop(columns=["pair"], inplace=True)
    return unique_pair_sums

In [21]:
def sigmoid_weight_transform(df, turning_point=2, col="tx_count"):
    df["weight"] = df[col].apply(lambda x: stretched_sigmoid(x, 2 / turning_point))
    return df

In [22]:
weight_df = columns_item_unique_pair_sums(native_trans_df_0)
weight_df = sigmoid_weight_transform(weight_df, 5)
weight_df.head()

,tx_count,node_a,node_b,weight
0,2,0x00000002f32c0886ee65d68059fbdb76ef6a6996,0x04f23cfd959b1f8aaf492e1d44c4c567102cf7db,0.689974
1,3,0x00000002f32c0886ee65d68059fbdb76ef6a6996,0x7fd06d484a0d6ae974c3b45e1e63ff9ee5160889,0.768525
2,1,0x00000002f32c0886ee65d68059fbdb76ef6a6996,0x8888889e0adf6c5afe2d5a6af91d31984365a1aa,0.598688
3,4,0x00000002f32c0886ee65d68059fbdb76ef6a6996,0xe43888320d5361ce4357d0dccdb2aee4ad99b9a0,0.832018
4,3,0x00000002f32c0886ee65d68059fbdb76ef6a6996,0xf36e5c04cdb9b465ce44e125c679f9d7a7fc799a,0.768525


# build community

In [23]:
def louvain_community_lst_func(
    df, resolution=1, node_cols=["node_a", "node_b"], weight_col="weight"
):
    unique_wallets_num = len(columns_unique_items(df, node_cols))
    num_edges = df.shape[0]
    print(f"Number of unique wallets: {unique_wallets_num}")
    print(f"Number of edges: {num_edges}")

    # Create a graph
    G = nx.Graph()

    # Add edges to the graph
    for _, row in df.iterrows():
        G.add_edge(row[node_cols[0]], row[node_cols[1]], weight=row[weight_col])

    communities = algorithms.louvain(G, weight=weight_col, resolution=resolution)

    communities_list = communities.communities
    print(f"Number of communities detected: {len(communities_list)}")
    print("-")
    display(pd.Series([len(community) for community in communities_list]).describe())
    print("")

    communities_list = sorted(communities_list, key=lambda x: len(x), reverse=True)

    return communities_list

In [24]:
def nested_lst_to_loc_dict(nested_lst_):
    loc_dict = {}
    for i, commu in enumerate(nested_lst_):
        for w in commu:
            loc_dict[w] = i
    return loc_dict

In [25]:
louvain_community_lst = louvain_community_lst_func(weight_df, resolution=200)

Number of unique wallets: 25198
Number of edges: 23489
Number of communities detected: 7103
-


count    7103.000000
mean        3.547515
std         5.146977
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max       102.000000
dtype: float64

In [26]:
def louvain_community_df_func(louvain_community_lst_):
    louvain_community_df_ = nested_lst_to_loc_dict(louvain_community_lst_)
    louvain_community_df_ = pd.DataFrame.from_dict(
        louvain_community_df_, orient="index", columns=["community"]
    )
    louvain_community_df_["type"] = "normal"
    return louvain_community_df_

In [27]:
louvain_community_df = louvain_community_df_func(louvain_community_lst)
louvain_community_df

,community,type
0x031c605262ec85aec6f793a5dd13bf21b6464af8,0,normal
0xb5a260659533ed9fe659b8c4254e4ff37047eadd,0,normal
0x04531b9d88da0eb58bb379637d4a914bdc122fdf,0,normal
0x0886b7d1549b70144eee26f05cf937259a3002d7,0,normal
0x112bcd74c8664cedbc0357f0021dfd485c891393,0,normal
...,...,...
0xfd6c69023c610350d6135b190ffb15b04a7aa9f2,7098,normal
0xfe603af99a456eff9a34cd9b29e5a5db9ffd4927,7099,normal
0xfe8da89113f28f65d71ee7b79de3b65704784529,7100,normal
0xff60fa1389cafb68b0537d69c981f5c4409977e7,7101,normal


### plot a community

In [57]:
plot_community_lst = louvain_community_lst[10]
plot_community_df = weight_df[
    weight_df["node_a"].isin(plot_community_lst)
    & weight_df["node_b"].isin(plot_community_lst)
]
print(plot_community_df.shape)
plot_community_df

(48, 4)


,tx_count,node_a,node_b,weight
1499,1,0x06209554bc06b2bb2a12446c0f0c741da532789c,0xa18df49ebc121d2bcf756b40f1b97ce8b835830f,0.598688
2174,1,0x09ef42ac46bd91d3039f58ac1c055239c65362c8,0xa18df49ebc121d2bcf756b40f1b97ce8b835830f,0.598688
2436,1,0x0ba57409d04c154b6bc931ea6cee199e3b12444a,0xa18df49ebc121d2bcf756b40f1b97ce8b835830f,0.598688
3636,1,0x10befa88586c8a378849dfe932c59939e8548875,0xa18df49ebc121d2bcf756b40f1b97ce8b835830f,0.598688
4619,1,0x167479c2d37f5bd4259acaef05e9ca7c948e7eff,0xa18df49ebc121d2bcf756b40f1b97ce8b835830f,0.598688
9509,1,0x36017ca03dc27cf5ecd3545e6cfda790abe64a69,0xa18df49ebc121d2bcf756b40f1b97ce8b835830f,0.598688
9580,1,0x366d0f389109e84209924b444182d052d8ae03ea,0xa18df49ebc121d2bcf756b40f1b97ce8b835830f,0.598688
10646,1,0x3c1039f91726c25cb859782b1caa9eafd6bf0e63,0xa18df49ebc121d2bcf756b40f1b97ce8b835830f,0.598688
11132,1,0x400cd82b8c2565fec580ac0910ed2f1241bcaa97,0xa18df49ebc121d2bcf756b40f1b97ce8b835830f,0.598688
12411,1,0x4b3483c855d223141cc6f2a1b1dc82e6618f52ef,0xa18df49ebc121d2bcf756b40f1b97ce8b835830f,0.598688


In [29]:
def community_visualization_2d(
    df,
    highlight_lst,
    highlight_lst_2_=None,
    source_col="node_a",
    target_col="node_b",
    weight_col="weight",
    weight_=800,
    height_=800,
):
    # Create a directed graph
    Gt = nx.from_pandas_edgelist(
        df,
        source=source_col,
        target=target_col,
        edge_attr=weight_col,
        create_using=nx.DiGraph(),
    )

    # Generate 2D positions for nodes
    pos = nx.spring_layout(Gt, weight=weight_col)

    # Extract node and edge positions
    edge_x = []
    edge_y = []
    edge_text = []  # List to hold weight information for hover
    for edge in Gt.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_text.append(f"weight: {round(edge[2][weight_col], 2)}")

    node_x = [pos[node][0] for node in Gt.nodes()]
    node_y = [pos[node][1] for node in Gt.nodes()]

    # Determine node colors based on the DataFrame
    node_colors = []
    for node in Gt.nodes():
        if node in highlight_lst:
            node_colors.append("yellowgreen")  # True condition
        elif highlight_lst_2_ and node in highlight_lst_2_:
            node_colors.append("dodgerblue")  # False condition
        else:
            node_colors.append("salmon")

    # Create Plotly figure
    fig = go.Figure()

    # Add edges
    fig.add_trace(
        go.Scatter(
            x=edge_x,
            y=edge_y,
            mode="lines",
            line=dict(color="black", width=2),
            hoverinfo="text",
            text=edge_text,  # Add edge weights to hover info
        )
    )

    # Add nodes
    fig.add_trace(
        go.Scatter(
            x=node_x,
            y=node_y,
            mode="markers",
            marker=dict(size=10, color=node_colors, line=dict(width=1, color="gray")),
            text=list(Gt.nodes()),  # Show node labels
            hoverinfo="text",
        )
    )

    # Update layout
    fig.update_layout(
        showlegend=False,
        width=weight_,
        height=height_,
        xaxis=dict(showgrid=True, zeroline=False),
        yaxis=dict(showgrid=False, zeroline=False),
        # plot_bgcolor="lightgray",
        title="Community Visualization",
    )

    # Show the plot
    fig.show()

In [58]:
community_visualization_2d(plot_community_df, core_wallet_lst, weight_=800, height_=500)

In [31]:
def community_visualization_3d(
    df,
    highlight_lst,
    highlight_lst_2_=None,
    source_col="node_a",
    target_col="node_b",
    weight_col="weight",
    weight_=800,
    height_=800,
):
    # Create a directed graph
    Gt = nx.from_pandas_edgelist(
        df,
        source=source_col,
        target=target_col,
        edge_attr=weight_col,
        create_using=nx.DiGraph(),
    )

    # Generate 3D positions
    pos = nx.spring_layout(Gt, dim=3, weight=weight_col)

    # Extract node and edge positions
    edge_x = []
    edge_y = []
    edge_z = []
    edge_text = []  # List to hold weight information for hover
    for edge in Gt.edges(data=True):
        x0, y0, z0 = pos[edge[0]]
        x1, y1, z1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])
        edge_text.append(f"weight: {edge[2][weight_col]}")

    node_x = [pos[node][0] for node in Gt.nodes()]
    node_y = [pos[node][1] for node in Gt.nodes()]
    node_z = [pos[node][2] for node in Gt.nodes()]

    # Determine node colors based on the DataFrame
    node_colors = []
    for node in Gt.nodes():
        if node in highlight_lst:
            node_colors.append("yellowgreen")  # True condition
        elif highlight_lst_2_ and node in highlight_lst_2_:
            node_colors.append("dodgerblue")  # False condition
        else:
            node_colors.append("salmon")

    # Create Plotly figure
    fig = go.Figure()

    # Add edges
    fig.add_trace(
        go.Scatter3d(
            x=edge_x,
            y=edge_y,
            z=edge_z,
            mode="lines",
            line=dict(color="black", width=2),
            text=edge_text,
            hoverinfo="text",
        )
    )

    # Add nodes
    fig.add_trace(
        go.Scatter3d(
            x=node_x,
            y=node_y,
            z=node_z,
            mode="markers",
            marker=dict(size=5, color=node_colors),
            text=list(Gt.nodes()),  # Show node labels on hover
            hoverinfo="text",
            textfont=dict(size=5),
            # hoverinfo="none",
        )
    )

    axis_setting = dict(
        showbackground=True,
        titlefont=dict(size=10),
        tickfont=dict(size=10),
    )
    # Update layout
    fig.update_layout(
        showlegend=False,
        width=weight_,
        height=height_,
        scene=dict(
            xaxis=axis_setting,
            yaxis=axis_setting,
            zaxis=axis_setting,
        ),
    )

    # Show the plot
    fig.show()

In [32]:
community_visualization_3d(plot_community_df, core_wallet_lst, weight_=600, height_=600)

# Get clustering variables

## Community Data

Calculate how many wallets each wallet has interacted with within the community

In [33]:
def columns_item_sum(df_, cols=["node_a", "node_b"], sum_col="tx_count"):
    all_elements = pd.concat(
        [
            df_[[cols[0], sum_col]].rename(columns={cols[0]: "index"}),
            df_[[cols[1], sum_col]].rename(columns={cols[1]: "index"}),
        ]
    )
    
    # Group by wallet and sum the transaction count
    df_sum_ = all_elements.groupby("index")[sum_col].sum().reset_index()
    return df_sum_

In [34]:
def clustering_community_info(w_lst, weight_df_=weight_df):
    internal_weight_df = weight_df_[
        weight_df_["node_a"].isin(w_lst) & weight_df_["node_b"].isin(w_lst)
    ]
    # get the number of wallets each wallet interacts with
    wallet_internal_tx_wallet_df = columns_item_count(
        internal_weight_df, w_cols=["node_a", "node_b"]
    )
    wallet_internal_tx_wallet_df.columns = ["wallet", "commu_tx_wallets"]
    wallet_internal_tx_wallet_df = wallet_internal_tx_wallet_df.set_index("wallet")
    # Get the sum of transactions for each wallet
    wallet_internal_tx_count_df = columns_item_sum(
        internal_weight_df, cols=["node_a", "node_b"], sum_col="tx_count"
    )
    wallet_internal_tx_count_df.columns = ["wallet", "commu_tx_count"]
    wallet_internal_tx_count_df = wallet_internal_tx_count_df.set_index("wallet")
    # merge the two dataframes
    wallet_internal_interaction_df = wallet_internal_tx_wallet_df.join(
        wallet_internal_tx_count_df
    )
    return wallet_internal_interaction_df

## Trino data 

"xxx_address_monthly_stats_full"
- address active days
- total gas fee in usd
- number of transaction
- wallets interacted
- last transaction time

-- 

(function is done but not sure if should use due to computational power required)

"xxx_address_last_stats_full"
- number of token types on different chains (ETH on Polygon and ETH on ethereum are different)

--

"xxx_token_transfers_full" (not yet given the computational power required)
- interval in second




In [35]:
def clustering_trino_info_fetch(w_lst, start_date_, end_date_):
    clustering_info = wallets_monthly_info(w_lst, start_date_, end_date_)
    clustering_info_df = pd.DataFrame(clustering_info).T
    clustering_info_df.columns = trino_data_column_names
    return clustering_info_df

In [36]:
def clustering_trino_info(w_lst, start_date_, end_date_, existing_df_=None, write_to_path=None):
    if existing_df_ is not None:
        existing_df_ = existing_df_[existing_df_.index.isin(w_lst)]
        # fetch for the missing wallets
        missing_lst = refine_lst(w_lst, existing_df_.index.tolist())
        if missing_lst:
            missing_df = clustering_trino_info_fetch(missing_lst, start_date_, end_date_)
            if write_to_path:
                write_to_file_name = os.path.join(write_to_path, f"clustering_trino_{missing_df.index[0]}.csv")
                missing_df.to_csv(write_to_file_name, index=True)
            return pd.concat([existing_df_, missing_df])
        else:
            return existing_df_
    else:
        return clustering_trino_info_fetch(w_lst, start_date_, end_date_)

### Combine data from two sources

In [37]:
def clustering_info(w_lst, start_date_, end_date_, existing_df_ = None, write_to_path=None):
    internal_info = clustering_community_info(w_lst)
    trino_info = clustering_trino_info(w_lst, start_date_, end_date_, existing_df_ = existing_df_, write_to_path=write_to_path)
    clustering_info_df = internal_info.merge(
        trino_info, left_index=True, right_index=True
    )

    return clustering_info_df

In [38]:
def full_community_clustering_info(louvain_community_lst_, start_date_, end_date_):
    for _, commu in enumerate(louvain_community_lst_):
        if len(commu) < 3:
            continue
        clustering_info_df = clustering_info(commu, start_date_, end_date_)
        file_path = f"/Users/floras/Desktop/unique-user/-Data/clustering_info/clustering_info_{start_date_}_{end_date_}.csv"
        if os.path.isfile(file_path):
            # Append to the file without writing the header
            clustering_info_df.to_csv(file_path, mode="a", header=False, index=True)
        else:
            # Write a new file with the header
            clustering_info_df.to_csv(file_path, mode="w", header=True, index=True)

# clustering

**Why we scale data?**

- If the dataset has features with vastly different scales (e.g., one feature ranges from 0 to 1, while another ranges from 0 to 1000), the feature with the larger scale will dominate the distance calculations. Normalizing data ensures that each feature contributes equally to the distance calculation. This is particularly important in multi-dimensional datasets where features may not have the same units or magnitude.

- we used standardization (Z-Score Scaling): Scales features to have mean = 0 and standard deviation = 1.

In [39]:
def isolation_forest(clustering_info_df, clustring_features, contamination_):
    if contamination_ > 0.5:
        clustering_info_df["isolation_forest"] = -1
        return clustering_info_df
    # Normalize the data
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(clustering_info_df[clustring_features])

    # Apply Isolation Forest
    iso = IsolationForest(contamination=contamination_)  # Adjust contamination level
    clustering_info_df["isolation_forest"] = iso.fit_predict(scaled_features)

    return clustering_info_df

In [40]:
def optics_clustering(clustering_info_df, clustring_features, min_samples_=3):
    if len(clustering_info_df) < min_samples_:
        clustering_info_df["optics"] = 0
        return clustering_info_df
    # Normalize the data
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(clustering_info_df[clustring_features])

    # Apply OPTICS clustering
    optics = OPTICS(min_samples=min_samples_)
    clustering_info_df["optics"] = optics.fit_predict(scaled_features)

    return clustering_info_df

In [41]:
def two_step_clustering(
    clustering_info_df,
    iso_features_,
    optics_features_,
    core_num=2,
    optics_min_samples=3,
):
    core_num = min(core_num, np.floor(clustering_info_df.shape[0] / 2))
    if core_num < 1:
        raise ValueError("Core number should be at least 1")
    contamination_ = core_num / clustering_info_df.shape[0]
    clustered_df = clustering_info_df.copy()
    clustered_df = isolation_forest(clustered_df, iso_features_, contamination_)
    middle_info_df = clustered_df[clustered_df["isolation_forest"] != -1]
    clusted_middle_info_df = optics_clustering(
        middle_info_df, optics_features_, optics_min_samples
    )
    # merge result back to original df
    clustered_df = pd.merge(
        clustered_df,
        clusted_middle_info_df["optics"],
        how="left",
        left_index=True,
        right_index=True,
    )
    clustered_df["type"] = clustered_df.apply(
        lambda x: (
            "sybil_core"
            if x["isolation_forest"] == -1
            else "normal" if x["optics"] == -1 else "sybil"
        ),
        axis=1,
    )
    return clustered_df

In [61]:
existing_file_path = '-Data/clustering_info'
existing_file_name_lst = file_name_lst(existing_file_path, endswith=".csv")
existing_data_df = read_from_multiple_csv(existing_file_path, existing_file_name_lst).set_index("Unnamed: 0")
existing_data_df = existing_data_df[trino_data_column_names]

existing_data_df.head()

,active_days,total_gas_fee_in_usd,txn_count,wallets_interacted,last_txn_date_int,last_txn_date,start_date,end_date
Unnamed: 0,,,,,,,,
0x1d26dcab017832a2336db5bc3218f99eb3807d57,88,621.600245,110,47,738875,2024-12-07,NaN,NaN
0xf3c17da125a33de55767aad78747a373200f3b98,59,237.204729,68,28,739221,2024-12-01,NaN,NaN
0xc700e16f61041fcf717f0930227e26948c746bc5,56,135.499519,68,32,738857,2024-12-01,NaN,NaN
0x28134b0c8b407fde3ba6edd3f9a39d4729927fb6,5,6.768358,5,2,738915,2024-01-30,NaN,NaN
0x95ee8b32aac20a4ca806d801774fc5e31fece7d4,208,1060.295574,244,109,739193,2024-12-10,NaN,NaN


In [60]:
test_df = clustering_info(plot_community_lst, None, None, existing_data_df, write_to_path=existing_file_path)
test_df

,commu_tx_wallets,commu_tx_count,active_days,total_gas_fee_in_usd,txn_count,wallets_interacted,last_txn_date_int,last_txn_date,start_date,end_date
0x06209554bc06b2bb2a12446c0f0c741da532789c,1,1,247,85.908282,265,52,739225,2024-12-05,None,None
0xa18df49ebc121d2bcf756b40f1b97ce8b835830f,48,48,7437,9.210677,7437,7436,739093,2024-07-26,None,None
0x09ef42ac46bd91d3039f58ac1c055239c65362c8,1,1,5,2.535241,5,5,738910,2024-07-12,None,None
0x0ba57409d04c154b6bc931ea6cee199e3b12444a,1,1,4,1.12019,4,4,738904,2024-07-13,None,None
0x10befa88586c8a378849dfe932c59939e8548875,1,1,5,1.790626,5,5,738910,2024-07-12,None,None
0x167479c2d37f5bd4259acaef05e9ca7c948e7eff,1,1,20,3.812898,26,16,738931,2024-09-14,None,None
0x36017ca03dc27cf5ecd3545e6cfda790abe64a69,1,1,2,0.00559,2,2,738904,2024-01-19,None,None
0x366d0f389109e84209924b444182d052d8ae03ea,1,1,4,3.068084,4,4,738904,2024-10-30,None,None
0x3c1039f91726c25cb859782b1caa9eafd6bf0e63,1,1,94,418.761305,134,64,739000,2024-12-12,None,None
0x400cd82b8c2565fec580ac0910ed2f1241bcaa97,1,1,1,0.000593,1,1,738904,2024-01-19,None,None


In [64]:
louvain_community_df

,community,type
0x031c605262ec85aec6f793a5dd13bf21b6464af8,0,normal
0xb5a260659533ed9fe659b8c4254e4ff37047eadd,0,normal
0x04531b9d88da0eb58bb379637d4a914bdc122fdf,0,normal
0x0886b7d1549b70144eee26f05cf937259a3002d7,0,normal
0x112bcd74c8664cedbc0357f0021dfd485c891393,0,normal
...,...,...
0xfd6c69023c610350d6135b190ffb15b04a7aa9f2,7098,normal
0xfe603af99a456eff9a34cd9b29e5a5db9ffd4927,7099,normal
0xfe8da89113f28f65d71ee7b79de3b65704784529,7100,normal
0xff60fa1389cafb68b0537d69c981f5c4409977e7,7101,normal


In [62]:
iso_features = ["commu_tx_wallets"]
optics_features = [
    "active_days",
    "total_gas_fee_in_usd",
    "txn_count",
    "wallets_interacted",
    "last_txn_date_int",
]

clustered_df = two_step_clustering(
    test_df, iso_features, optics_features, core_num = 5, optics_min_samples = 3
)
print(len(clustered_df))
clustered_df.sort_values(by=['optics'])

49


,commu_tx_wallets,commu_tx_count,active_days,total_gas_fee_in_usd,txn_count,wallets_interacted,last_txn_date_int,last_txn_date,start_date,end_date,isolation_forest,optics,type
0x06209554bc06b2bb2a12446c0f0c741da532789c,1,1,247,85.908282,265,52,739225,2024-12-05,None,None,1,-1.0,normal
0x6d03833ed2d9d06dcd27c3b9ca19b84cca97bf37,1,1,102,366.278991,128,58,739026,2024-12-03,None,None,1,-1.0,normal
0x6b6612149a94e04c0d90bc5699ea91e1f862db2f,1,1,26,52.317356,27,21,738930,2024-11-12,None,None,1,-1.0,normal
0xd2d9b20d9847a7c96c93c385c8d6e85164700470,1,1,158,2797.19979,352,73,738920,2024-09-28,None,None,1,-1.0,normal
0x8ba8aa2e7589b8f89c212ea03fea86219b1e3558,1,1,1,0.000612,1,1,738905,2024-01-20,None,None,1,-1.0,normal
0x5911035a34855c21dbc1fddf544946c360a43495,1,1,417,17.926255,820,67,739232,2024-12-12,None,None,1,-1.0,normal
0x53c0d3811f103200e44f39a0f523046c3116a6dd,1,1,3,0.003929,3,3,738941,2024-02-25,None,None,1,-1.0,normal
0xc1bef26ba79174e03d82433384608dc3be04e585,1,1,1,0.000485,1,1,738908,2024-01-23,None,None,1,-1.0,normal
0x9c180ea0636010aa85d97b23381f5a7401054b21,1,1,2,0.002713,2,2,739061,2024-06-24,None,None,1,-1.0,normal
0xdb2620c92f8e0aef798572ef82dd672040f69635,1,1,19,9.014891,39,12,738945,2024-09-29,None,None,1,-1.0,normal


In [63]:
clustered_core_wallets = clustered_df[
    clustered_df["type"] == "sybil_core"
].index.tolist()
clustered_normal_wallets = clustered_df[clustered_df["type"] == "normal"].index.tolist()

community_visualization_2d(
    plot_community_df,
    clustered_core_wallets,
    clustered_normal_wallets,
    weight_=600,
    height_=600,
)

In [232]:
ttt = louvain_community_df.copy()
ttt["type"] = "normal"
ttt["type"].update(clustered_df["type"])

In [233]:
ttt["type"].value_counts()

normal        25194
sybil             3
sybil_core        1
Name: type, dtype: int64